In [2]:
import random

class Carta:
    
    naipes = ['♠️', '♥️', '♦️', '♣️']
    valores = [None, 'Ás', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'Valete', 'Dama', 'Rei']
        
    def __init__(self, valor=2, naipe=0):
        self.valor = valor
        self.naipe = naipe

    def __str__(self):
        return f"{Carta.valores[self.valor]}{Carta.naipes[self.naipe]}"
    
    def __lt__(self, other):
        t1 = self.valor, self.naipe
        t2 = other.valor, other.naipe
        return t1 < t2

class Baralho:
    def __init__(self):
        self.cartas = []
        for naipe in range(4):
            for valor in range(1,14):
                self.cartas.append(Carta(valor, naipe))
    
    def __str__(self):
        cartas_str = []
        for carta in self.cartas:
            cartas_str.append(str(carta))
        return ', '.join(cartas_str)
    
    def pop_carta(self):
        return self.cartas.pop()
    
    def add_carta(self, carta):
        self.cartas.append(carta)

    def embaralhar(self):
        random.shuffle(self.cartas)
        
    def move_cartas(self, mao, num):
        for i in range(num):
            mao.add_carta(self.pop_carta())
            
class Mao(Baralho):
    def __init__(self, label=''):
        self.cartas = []
        self.label = label

class Poker:
    def __init__(self):
        self.baralho = Baralho()
        self.baralho.embaralhar()
        #print(self.baralho)
        self.jogador1 = Mao("Jogador 1")
        self.jogador2 = Mao("Jogador 2")
        
    def par(self, jogador):
        valores = [carta.valor for carta in jogador.cartas]
        for valor in valores:
            if valores.count(valor) == 2:
                return True
        return False

    def dois_pares(self, jogador):
        valores = [carta.valor for carta in jogador.cartas]
        pares = 0
        for valor in set(valores):
            if valores.count(valor) == 2:
                pares += 1
        return pares == 2
    
    def trinca(self, jogador):
        valores = [carta.valor for carta in jogador.cartas]
        for valor in valores:
            if valores.count(valor) == 3:
                return True
        return False
    
    def sequencia(self, jogador):
        valores = sorted([carta.valor for carta in jogador.cartas])
        for i in range(len(valores) - 1):
            if valores[i + 1] - valores[i] != 1:
                return False
        return True
    
    def flush(self, jogador):
        naipes = [carta.naipe for carta in jogador.cartas]
        return len(set(naipes)) == 1
    
    def full_house(self, jogador):
        valores = [carta.valor for carta in jogador.cartas]
        valores_count = {valor: valores.count(valor) for valor in set(valores)}
        return len(set(valores)) == 2 and 3 in valores_count.values() and 2 in valores_count.values()
    
    def quadra(self, jogador):
        valores = [carta.valor for carta in jogador.cartas]
        for valor in valores:
            if valores.count(valor) == 4:
                return True
        return False
    
    def straight_flush(self, jogador):
        return self.flush(jogador) and self.sequencia(jogador)
    
    def royal_flush(self, jogador):
        valores = sorted([carta.valor for carta in jogador.cartas])
        naipes = [carta.naipe for carta in jogador.cartas]
        return valores == [10, 11, 12, 13, 1] and len(set(naipes)) == 1

    def distribuir_cartas(self):
        for _ in range(5):
            self.baralho.move_cartas(self.jogador1, 1)
            self.baralho.move_cartas(self.jogador2, 1)
    
    def trocar_cartas(self, jogador, cartas_trocar):
        if cartas_trocar < 0:
            cartas_trocar = 0
        elif cartas_trocar > 2:
            print("É possível trocar até no máximo 2 cartas.")
            cartas_trocar = 2

        if cartas_trocar != 0: 
            while True:
                try:
                    trocar = [int(x) - 1 for x in input(f"Digite quais cartas o {jogador.label} deseja trocar? (1-5, separadas por espaço): ").split()]
                    if len(trocar) != cartas_trocar:
                        print(f"Você precisa selecionar exatamente {cartas_trocar} cartas.")
                        continue
                    if all(0 <= indice < len(jogador.cartas) for indice in trocar):
                        break
                    else:
                        print("Fora do intervalo. Digite novamente.")
                except ValueError:
                    print("Digite um número válido.")

            trocadas = []
            for indice in sorted(trocar, reverse=True):
                carta_trocada = jogador.cartas.pop(indice)
                trocadas.append(carta_trocada)
            for _ in range(cartas_trocar):
                jogador.add_carta(self.baralho.pop_carta())

    
    def mostrar_cartas(self):
        print(f"{self.jogador1.label}: {', '.join(map(str, self.jogador1.cartas))}")
        print(f"{self.jogador2.label}: {', '.join(map(str, self.jogador2.cartas))}")
        
    def jogador_inicial(self):
        carta_jogador1 = random.choice(self.jogador1.cartas)
        carta_jogador2 = random.choice(self.jogador2.cartas)
        print(f"Primeira carta do jogador 1: {carta_jogador1}\nPrimeira carta do jogador 2: {carta_jogador2}\n")

        if carta_jogador1 > carta_jogador2:
            return self.jogador1
        else:
            return self.jogador2
    
    def comparar_maos(self, aux1, aux2, jogador1, jogador2):
        maos = {
            'royal_flush': 9,
            'straight_flush': 8,
            'quadra': 7,
            'full_house': 6,
            'flush': 5,
            'sequencia': 4,
            'trinca': 3,
            'dois_pares': 2,
            'par': 1
        }

        if aux1 == 'dois_pares' and aux2 == 'dois_pares':
            cartas_jogador1 = sorted([carta for carta in jogador1.cartas if jogador1.cartas.count(carta) == 1], reverse=True)
            cartas_jogador2 = sorted([carta for carta in jogador2.cartas if jogador2.cartas.count(carta) == 1], reverse=True)
            
            for carta1, carta2 in zip(cartas_jogador1, cartas_jogador2):
                if carta1 != carta2:
                    return "Jogador 1" if carta1 > carta2 else "Jogador 2"
            return "Empate"

        if aux1 == 'par' and aux2 == 'par':
            cartas_jogador1 = [carta.valor for carta in jogador1.cartas if jogador1.cartas.count(carta) != 2]
            cartas_jogador2 = [carta.valor for carta in jogador2.cartas if jogador2.cartas.count(carta) != 2]

            cartas_jogador1.sort(reverse=True)
            cartas_jogador2.sort(reverse=True)
            
            for carta_jogador1, carta_jogador2 in zip(cartas_jogador1, cartas_jogador2):
                if carta_jogador1 != carta_jogador2:
                    return "Jogador 1" if carta_jogador1 > carta_jogador2 else "Jogador 2"
            return "Empate"
        
        if aux1 == 'trinca' and aux2 == 'trinca':
            cartas_jogador1 = sorted([carta for carta in jogador1.cartas if jogador1.cartas.count(carta) != 3], reverse=True)
            cartas_jogador2 = sorted([carta for carta in jogador2.cartas if jogador2.cartas.count(carta) != 3], reverse=True)
            
            for carta1, carta2 in zip(cartas_jogador1, cartas_jogador2):
                if carta1 != carta2:
                    return "Jogador 1" if carta1 > carta2 else "Jogador 2"
            
            for carta1, carta2 in zip(cartas_jogador1, cartas_jogador2):
                if carta1 != carta2:
                    return "Jogador 1" if carta1 > carta2 else "Jogador 2"
            return "Empate"

        if aux1 == 'sequencia' and aux2 == 'sequencia':
            carta_alta_jogador1 = max(jogador1.cartas)
            carta_alta_jogador2 = max(jogador2.cartas)
            
            if carta_alta_jogador1 != carta_alta_jogador2:
                return "Jogador 1" if carta_alta_jogador1 > carta_alta_jogador2 else "Jogador 2"
            else:
                return "Empate"

        if aux1 == 'flush' and aux2 == 'flush':
            carta_alta_jogador1 = max([carta for carta in jogador1.cartas if carta.naipe == jogador1.cartas[0].naipe])
            carta_alta_jogador2 = max([carta for carta in jogador2.cartas if carta.naipe == jogador2.cartas[0].naipe])
            
            if carta_alta_jogador1 != carta_alta_jogador2:
                return "Jogador 1" if carta_alta_jogador1 > carta_alta_jogador2 else "Jogador 2"
            else:
                return "Empate"

        if aux1 == 'full_house' and aux2 == 'full_house':
            cartas_trinca_jogador1 = [carta.valor for carta in jogador1.cartas if jogador1.cartas.count(carta) == 3]
            cartas_trinca_jogador2 = [carta.valor for carta in jogador2.cartas if jogador2.cartas.count(carta) == 3]
            
            for carta1, carta2 in zip(sorted(cartas_trinca_jogador1, reverse=True), sorted(cartas_trinca_jogador2, reverse=True)):
                if carta1 != carta2:
                    return "Jogador 1" if carta1 > carta2 else "Jogador 2"
                
            return "Empate"

        if aux1 == 'quadra' and aux2 == 'quadra':
            carta_restante_jogador1 = max([carta.valor for carta in jogador1.cartas if jogador1.cartas.count(carta) != 4])
            carta_restante_jogador2 = max([carta.valor for carta in jogador2.cartas if jogador2.cartas.count(carta) != 4])
            
            if carta_restante_jogador1 != carta_restante_jogador2:
                return "Jogador 1" if carta_restante_jogador1 > carta_restante_jogador2 else "Jogador 2"
            else:
                return "Empate"

        if aux1 == 'straight_flush' and aux2 == 'straight_flush':
            carta_mais_alta_jogador1 = max([carta.valor for carta in jogador1.cartas])
            carta_mais_alta_jogador2 = max([carta.valor for carta in jogador2.cartas])
            
            if carta_mais_alta_jogador1 != carta_mais_alta_jogador2:
                return "Jogador 1" if carta_mais_alta_jogador1 > carta_mais_alta_jogador2 else "Jogador 2"
            else:
                return "Empate"

        if aux1 == 'royal_flush' and aux2 == 'royal_flush':
            naipe_maior_jogador1 = max([carta.naipe for carta in jogador1.cartas])
            naipe_maior_jogador2 = max([carta.naipe for carta in jogador2.cartas])
            
            if naipe_maior_jogador1 != naipe_maior_jogador2:
                return "Jogador 1" if naipe_maior_jogador1 > naipe_maior_jogador2 else "Jogador 2"
            else:
                return "Empate"

        if aux1 == '' and aux2 == '':
            carta_alta_jogador1 = max(jogador1.cartas)
            carta_alta_jogador2 = max(jogador2.cartas)
            
            if carta_alta_jogador1.valor == carta_alta_jogador2.valor:
                if carta_alta_jogador1.naipe > carta_alta_jogador2.naipe:
                    return "Jogador 1"
                elif carta_alta_jogador1.naipe < carta_alta_jogador2.naipe:
                    return "Jogador 2"
                else:
                    return "Empate"
            elif carta_alta_jogador1.valor > carta_alta_jogador2.valor:
                return "Jogador 1"
            else:
                return "Jogador 2"
        elif aux1 == '':
            return "Jogador 2"
        elif aux2 == '':
            return "Jogador 1"

        if maos[aux1] > maos[aux2]:
            return "Jogador 1"
        elif maos[aux1] < maos[aux2]:
            return "Jogador 2"
        else:
            return "Empate"

    def jogar(self):
        print("\n\nDistribuição de cartas...\n")
        self.distribuir_cartas()
        self.mostrar_cartas()
        
        jogador_inicial = self.jogador_inicial()
        print(f"O jogador a iniciar é o \"{jogador_inicial.label}\"")
        
        if jogador_inicial == self.jogador2:
            while True:
                try:
                    trocasJogador2 = int(input("Jogador 2\nDigite quantas cartas deseja trocar (0-2)? "))
                    break
                except ValueError:
                    print("Digite um número válido.")
            self.trocar_cartas(self.jogador2, trocasJogador2)
            
            while True:
                try:
                    trocasJogador1 = int(input("Jogador 1\nDigite quantas cartas deseja trocar (0-2)? "))
                    break
                except ValueError:
                    print("Digite um número válido.")
            self.trocar_cartas(self.jogador1, trocasJogador1)
        else: 
            while True:
                try:
                    trocasJogador1 = int(input("Jogador 1\nDigite quantas cartas deseja trocar (0-2)? "))
                    break
                except (ValueError, IndexError):
                    print("Digite um número válido.")
            self.trocar_cartas(self.jogador1, trocasJogador1)
            
            while True:
                try:
                    trocasJogador2 = int(input("Jogador 2\nDigite quantas cartas deseja trocar (0-2)? "))
                    break
                except (ValueError, IndexError):
                    print("Digite um número válido.")
            self.trocar_cartas(self.jogador2, trocasJogador2)
        
        print("\nCartas após troca:")
        self.mostrar_cartas()
        aux1 = ''
        aux2 = ''
        
        if self.full_house(self.jogador1):
            print("Jogador 1 tem um full house!")
            aux1 = 'full_house'
        if self.trinca(self.jogador1):
            if(aux1 != 'full_house'):
                print("Jogador 1 tem uma trinca!")
                aux1 = 'trinca'
        elif self.dois_pares(self.jogador1):
            print("Jogador 1 tem dois pares!")
            aux1 = 'dois_pares'
        elif self.par(self.jogador1):
            print("Jogador 1 tem um par!")
            aux1 = 'par'
        if self.sequencia(self.jogador1):
            print("Jogador 1 tem uma sequência!")
            aux1 = 'sequencia'
        if self.flush(self.jogador1):
            print("Jogador 1 tem um flush!")
            aux1 = 'flush'
        if self.quadra(self.jogador1):
            print("Jogador 1 tem uma quadra!")
            aux1 = 'quadra'
        if self.straight_flush(self.jogador1):
            print("Jogador 1 tem um straight flush!")
            aux1 = 'straight_flush'
        if self.royal_flush(self.jogador1):
            print("Jogador 1 tem um royal flush!")
            aux1 = 'royal_flush'
        
        if self.full_house(self.jogador2):
            print("Jogador 2 tem um full house!")
            aux2 = 'full_house'
        if self.trinca(self.jogador2):
            if(aux2 != 'full_house'):
                print("Jogador 2 tem uma trinca!")
                aux2 = 'trinca'
        elif self.dois_pares(self.jogador2):
            print("Jogador 2 tem dois pares!")
            aux2 = 'dois_pares'
        elif self.par(self.jogador2):
            print("Jogador 2 tem um par!")
            aux2 = 'par'
        if self.sequencia(self.jogador2):
            print("Jogador 2 tem uma sequência!")
            aux2 = 'sequencia'
        if self.flush(self.jogador2):
            print("Jogador 2 tem um flush!")
            aux2 = 'flush'
        if self.quadra(self.jogador2):
            print("Jogador 2 tem uma quadra!")
            aux2 = 'quadra'
        if self.straight_flush(self.jogador2):
            print("Jogador 2 tem um straight flush!")
            aux2 = 'straight_flush'
        if self.royal_flush(self.jogador2):
            print("Jogador 2 tem um royal flush!")
            aux2 = 'royal_flush'

        
        resultado_jogo = self.comparar_maos(aux1, aux2, self.jogador1, self.jogador2)
        print(f"Vencedor: {resultado_jogo}")


jogo = Poker()
jogo.jogar()
while True:
    jogar_novamente = input("Deseja jogar novamente? (s/n): ").strip().lower()
    if jogar_novamente != 's':
        print("Obrigado por jogar!")
        break
    else:
        jogo = Poker()
        jogo.jogar()




Distribuição de cartas...

Jogador 1: 10♥️, 8♠️, 9♣️, 5♥️, Dama♥️
Jogador 2: 2♥️, 5♦️, 2♣️, 3♠️, 3♣️
Primeira carta do jogador 1: 9♣️
Primeira carta do jogador 2: 2♣️

O jogador a iniciar é o "Jogador 1"


KeyboardInterrupt: Interrupted by user